# Store Sales - Time Series Forecasting

Some Ideas
- Use up until 2017 as "train data"

EDA

Models
- Linear Regression, using features of:
store_nbr, family, sales, onpromotion (given - this is more global)
store cluster, store type, store city + state (given - stores csv global)
day of the week, oil price

x days ago sales for (same family, same store, same family + same store)

ignore holidays

### What is the most basic MVP?

EDA
quickly graph:
- aggregated sales per day (across all families)
- sales per day (each family separately)
- aggregated sales per day (each store separately)

- transactions per day (maybe these 2 useless)
- transactions per store

Features and model
- store nbr, family, sales, onpromotion
- x days ago sales (same fam, same store, same fam + store)

- store cluster, store type, store location

# EDA

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time


In [46]:
# Load data
dataPath = './data/'
train = pd.read_csv(dataPath + 'train.csv')
train = train.set_index('id')

stores = pd.read_csv(dataPath + 'stores.csv')
oil = pd.read_csv(dataPath + 'oil.csv')
holidays = pd.read_csv(dataPath + 'holidays_events.csv')

In [47]:
# Combine data into single df
# 1. Store --> match store to (city, state, type, cluster)
# 2. Oil --> match day to oil (fill in na with mean of surrounding days)

# Fill in oil values
oilFilled = oil.bfill()

combined = train.merge(stores, on='store_nbr')
combined = combined.merge(oilFilled, on='date')
combined.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14
1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14
2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14
3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14
4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14


In [ ]:
# # Graph sales per family
# familySales = combined.groupby(by=['date', 'family'])['sales'].sum().unstack()

# n = len(familySales.columns)
# ncols = 3
# nrows = (n+ncols-1)//ncols
# fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15,nrows*5))

# for i, colName in enumerate(familySales.columns[:n]):
#     currRow = i//ncols
#     currCol = i%ncols

#     familySales[[colName]].plot(rot=30, ax=axs[currRow, currCol])

In [49]:
# # Graph sales per store
# storeSales = combined.groupby(by=['date', 'store_nbr'])['sales'].sum().unstack()

# n = len(storeSales.columns)
# ncols = 3
# nrows = (n+ncols-1)//ncols
# fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15,nrows*5))

# for i, colName in enumerate(storeSales.columns[:n]):
#     currRow = i//ncols
#     currCol = i%ncols

#     storeSales[[colName]].plot(rot=30, ax=axs[currRow, currCol])

# Model

We now want "x days ago" data.

The first such data is "x days ago, what is the mean sales for my given family"
Out of curiosity, let us explore the different methods:

1. Pd.apply. Take in a row --> filter the entire df for the desired date & family --> output mean. (10 sec)
2. Pd.apply. Preprocesses to create a new df with (date, family) : mean. Take in row --> use the preprocessed value. (5 sec)
3. Merge. Preprocess to create a new df with (date, family). Merge df with this preprocessed df. (1 sec)

In [58]:
combined['date'] = pd.to_datetime(combined['date'])
combined['date'].dtype
smallCombined = combined[:10000]
smallCombined.describe()

,date,store_nbr,sales,onpromotion,cluster,dcoilwtico
count,10000,10000.000000,10000.000000,10000.0,10000.000000,10000.000000
mean,2013-01-03 21:38:00.960000,26.948200,169.577629,0.0,8.429100,93.124464
min,2013-01-01 00:00:00,1.000000,0.000000,0.0,1.000000,92.970000
25%,2013-01-02 00:00:00,14.000000,0.000000,0.0,4.000000,93.120000
50%,2013-01-03 00:00:00,27.000000,0.000000,0.0,8.000000,93.140000
75%,2013-01-07 00:00:00,39.000000,40.000000,0.0,13.000000,93.200000
max,2013-01-08 00:00:00,54.000000,11422.000000,0.0,17.000000,93.210000
std,NaN,15.197513,624.142229,0.0,4.662268,0.078461


In [ ]:
# Method 1: Create function to filter entire df, for each row
# Runtime:
# Combined 10k length: 5 sec
# Combined 20k length: 17 sec

def prevday_family_mean(row):
    prevDay = row['date'] - pd.Timedelta(days=1)
    mask = (smallCombined['date'] == prevDay) & (smallCombined['family'] == row['family'])
    if not mask.sum():
        return None
    return smallCombined[mask]['sales'].mean()

start_time = time.time()
smallCombined['familyMean_1day'] = smallCombined.apply(lambda x: prevday_family_mean(x), axis=1)
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

smallCombined.describe()

Execution time: 5.038007020950317 seconds


/var/folders/_c/j62lsd3n7x9_vq272kdsl3fc0000gn/T/ipykernel_69148/450447730.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallCombined['familyMean_1day'] = smallCombined.apply(lambda x: prevday_family_mean(x), axis=1)


,date,store_nbr,sales,onpromotion,cluster,dcoilwtico,familyMean_1day
count,10000,10000.000000,10000.000000,10000.0,10000.000000,10000.000000,6436.000000
mean,2013-01-03 21:38:00.960000,26.948200,169.577629,0.0,8.429100,93.124464,165.549588
min,2013-01-01 00:00:00,1.000000,0.000000,0.0,1.000000,92.970000,0.000000
25%,2013-01-02 00:00:00,14.000000,0.000000,0.0,4.000000,93.120000,0.000000
50%,2013-01-03 00:00:00,27.000000,0.000000,0.0,8.000000,93.140000,0.851852
75%,2013-01-07 00:00:00,39.000000,40.000000,0.0,13.000000,93.200000,67.384481
max,2013-01-08 00:00:00,54.000000,11422.000000,0.0,17.000000,93.210000,3741.111111
std,NaN,15.197513,624.142229,0.0,4.662268,0.078461,502.456439


In [ ]:
# Method 2: Preprocess and then use pd.apply --> get the preprocessed value
# Time:
# 10k length: 5 sec
# 20k length: 18 sec

def prevday_family_mean_2(row):
    prevDay = row['date'] - pd.Timedelta(day=1)
    return familySales.at[prevDay, row['family']]

start_time = time.time()
familySales = smallCombined.groupby(by=['date', 'family'])['sales'].sum().unstack()
smallCombined['familyMean_1day'] = smallCombined.apply(lambda x: prevday_family_mean(x), axis=1)
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

smallCombined.describe()
smallCombined.head()

Execution time: 17.305140256881714 seconds


/var/folders/_c/j62lsd3n7x9_vq272kdsl3fc0000gn/T/ipykernel_69148/2995257211.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallCombined['familyMean_1day'] = smallCombined.apply(lambda x: prevday_family_mean(x), axis=1)


,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,familyMean_1day
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14,NaN
1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14,NaN
2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14,NaN
3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14,NaN
4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14,NaN


In [ ]:
# Method 3: Using merge
start_time = time.time()
familySales = smallCombined.groupby(by=['date', 'family'])['sales'].sum().unstack()

smallCombined['familyMean_1day'] = smallCombined.apply(lambda x: prevday_family_mean(x), axis=1)
end_time = time.time()